In [2]:
#| hide
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import nltk

# Cleaning Data

Set data path

In [4]:
raw_data = '../data/raw-data'

Generate a random numpy seed

In [6]:
np.random.seed(500)

Read in CSV Datafiledata

In [5]:
data = "Maintenance_Text_data"
df = pd.read_csv(f"{raw_data}/{data}.csv",encoding='latin-1', header=0)

Replace any whitespace with NaN

In [9]:
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df.head()

,Unnamed: 0,c5,c119,c78,c80,c86,c77,c79,c81,c85,...,c137,c146,c148,c150,c154,c161,c163,c183,c191,remark
0,0,19750315005389A,TAILWHEEL COCKED RIGHT PRIOR TO takeoff,AU,V,NaN,Attempt Operation Wi,Other/Miscellaneous,Pilot-In-Command,NaN,...,NaN,UNDER 12501 LBS,MONOPLANE-HIGH WING/PARA WING,POWERED,NaN,NaN,NaN,NaN,NaN,TAILWHEEL COCKED RIGHT PRIOR TO takeoff
1,1,19750419011349A,TOW PLANE BECAME AIRBORNE THEN SETTLED STUDENT...,ME,V,NaN,Fail/Incorrect Use M,Other/Miscellaneous,Pilot-In-Command,NaN,...,NaN,UNDER 12501 LBS,KITE/SAILWING,NONPOWERED,NaN,NaN,NaN,NaN,NaN,TOW PLANE BECAME AIRBORNE THEN SETTLED STUDENT...
2,2,19751029037799A,"2ND ILS APCH,aircraft'S G/S INOP LOM TUNED TO ...",AU,Z,IP,Attempt Operation Wi,Weather,Pilot-In-Command,Improper Inst Proc T,...,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,NaN,NaN,NaN,"2ND ILS APCH,aircraft'S G/S INOP LOM TUNED TO ..."
3,3,19751209037899A,pilot NOTED SOFT right BRAKE PEDAL DRG TAXI TO...,AU,V,NaN,Attempt Operation Wi,Other/Miscellaneous,Pilot-In-Command,NaN,...,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,NaN,NaN,NaN,pilot NOTED SOFT right BRAKE PEDAL DRG TAXI TO...
4,4,19750818025579A,TAXI OFF HARD SFC DUE TFC right MAIN GR BROKE ...,AF,H,NaN,Improper Maintenance,Hidden Hazardous Obj,Pilot-In-Command,NaN,...,NaN,UNDER 12501 LBS,MONOPLANE-HIGH WING/PARA WING,POWERED,NaN,NaN,NaN,NaN,NaN,TAXI OFF HARD SFC DUE TFC right MAIN GR BROKE ...


Count total number of NaN entires and save to file

In [12]:
df.isna().sum()
na_count.to_csv(f'../NaN_Counts/{data}NaCount.csv')

Next we count the percentages of NaN entires for each column

In [15]:
total = df.shape[0]
acceptable_pct_100 = total
acceptable_pct_90 = total * .9
acceptable_pct_80 = total * .8
acceptable_pct_70 = total * .7
acceptable_pct_60 = total * .6
acceptable_pct_50 = total * .5
acceptable_pct_40 = total * .4
acceptable_pct_30 = total * .3
acceptable_pct_20 = total * .2
acceptable_pct_10 = total * .1
acceptable_pct_0 = 0

acceptable_columns = {
    '100': [],
    '90': [],
    '80': [],
    '70': [],
    '60': [],
    '50': [],
    '40': [],
    '30': [],
    '20': [],
    '10': [],
    '0': [],
    'full': []
}

for column in df.columns:
    column_counts = df[column].value_counts()
    na_count = df[column].isna().sum()
    na_series = pd.Series([na_count], index=['NaN'])  
    column_counts = column_counts.append(na_series) 
    column_counts.to_csv(f'../individualColumns/{0}.csv'.format(column))
    if na_count == acceptable_pct_100:
        acceptable_columns['100'].append(column)
    elif na_count > acceptable_pct_90:
        acceptable_columns['90'].append(column)
    elif na_count > acceptable_pct_80:
        acceptable_columns['80'].append(column)
    elif na_count > acceptable_pct_70:
        acceptable_columns['70'].append(column)
    elif na_count > acceptable_pct_60:
        acceptable_columns['60'].append(column)
    elif na_count > acceptable_pct_50:
        acceptable_columns['50'].append(column)
    elif na_count > acceptable_pct_40:
        acceptable_columns['40'].append(column)
    elif na_count > acceptable_pct_30:
        acceptable_columns['30'].append(column)
    elif na_count > acceptable_pct_20:
        acceptable_columns['20'].append(column)
    elif na_count > acceptable_pct_10:
        acceptable_columns['10'].append(column)
    elif na_count > acceptable_pct_0:
        acceptable_columns['0'].append(column)
    elif na_count == acceptable_pct_0:
        acceptable_columns['full'].append(column)
        
total_columns_in_dict = 0
for key, value in acceptable_columns.items():
    print(key,' : ',len(value))
    total_columns_in_dict += len(value)

100  :  0
90  :  4
80  :  3
70  :  1
60  :  2
50  :  0
40  :  2
30  :  1
20  :  0
10  :  0
0  :  12
full  :  10


/tmp/ipykernel_2618093/3519092763.py:33: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  column_counts = column_counts.append(na_series)
/tmp/ipykernel_2618093/3519092763.py:33: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  column_counts = column_counts.append(na_series)
/tmp/ipykernel_2618093/3519092763.py:33: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  column_counts = column_counts.append(na_series)
/tmp/ipykernel_2618093/3519092763.py:33: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  column_counts = column_counts.append(na_series)
/tmp/ipykernel_2618093/3519092763.py:33: FutureWarning: The series.append method is deprecated and w

Here's an example of the 90% columns

In [19]:
acceptable_columns['90']

['c97', 'c135', 'c137', 'c163']

In [23]:
df = pd.read_csv(f'../data/{data}.csv',encoding='latin-1', header=0)
for col in df.columns:
    if col in acceptable_columns['0']:
        pass
    elif col in acceptable_columns['full']:
        pass
    elif col == 'c77' or col == 'c78':
        pass
    else:
        df.drop(inplace = True, columns=col)
        
df.to_csv(f'../Processed_Datasets/{data}.csv')

In [24]:
Main = pd.read_csv(f"../Original_Datasets/Concatenated_Orig_data.csv",encoding='latin-1', header=0)
Title = pd.read_csv("../Original_Datasets/Afilelayout.csv",encoding='latin-1', header=0)

count = 0
for key in Title['columnname']:
    count += 1
    if key in Main:
        Main.rename(columns={key : Title['description'][count-1]}, inplace=True)

Main.drop(inplace = True, columns='Unnamed: 0')
Main.to_csv(f'./Original_Datasets/Concatenated_Orig_data_renamed.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../Original_Datasets/Concatenated_Orig_data.csv'